### Description:

In this task you will work with *hardware* AES-128 running on ESP32. This engine is started by protocol:

mbedtls_aes_init( &aes );

mbedtls_aes_setkey_enc( &aes, (const unsigned char*) key, 128 );

mbedtls_aes_crypt_ecb(&aes, MBEDTLS_AES_ENCRYPT, (const unsigned char*)input, output);

mbedtls_aes_free( &aes );

You have access to plaintexts, ciphertexts and associated traces.
Oscilloscope snapshots (see below) show an entire AES execution, i.e., mbedtls_aes_crypt_ecb process. 

You will have only the part related to encryption process, i.e., a part shown from 8 us to 8.5 us on the second oscilloscope snapshot. 

The measurements were performed with LeCroy WaveRunner Zi625 and a differential probe

### Tips:

A 'trace' is a side-channel information measured between the VCC and the GND pins of the WROOM module (ESP32). 

Hardware IPs leak significantlly smaller amount of information than CPU. However, they are still leaking.

In this example a trace is a 500 bytes array of int8 (the amplitude of the measured voltage). The trace containes entire AES-128 encryption process.

This array contains various encryption information measured with noise.

### Task:

You need to find the master key.


The master key is in the form of SCA{XXXXXXXXXXX}, where X is a ASCII printable symbol (you already know 5 key bytes).

### AES-128 encryption measured at high sampling rate

Green signal - a GPIO trigger used to synchronize all the traces

Red signal   - VCC voltage variation during AES-128 encryption


<img src="support/osc_snap_0.png">

<img src="support/osc_snap_1.png">

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
import binascii
import h5py
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib

import sca_training

%matplotlib inline

numtraces   = 1000000
numsamples  = 500
numfiles    = 50

numtraces_per_file = 20000
     
#----------------------------------------------------------------------------
# OUTPUTS:
#     ptexts   - input plaintexts converted to numpy array of uint8
#     ctexts   - resulted ciphertext converted to numpy array of uint8
#     traces   - a trace associated with encryption process
#----------------------------------------------------------------------------
def ReadData():
    global numtraces
    global numsamples
    global numfiles
    global numtraces_per_file
    
    traces = np.zeros((numtraces, numsamples), dtype = np.int8)
    ptexts = np.zeros((numtraces, 16), dtype = np.uint8)
    ctexts = np.zeros((numtraces, 16), dtype = np.uint8)
    
    print('Reading data...')
    for i_file in range(numfiles):
        h5file = r'../data/aes_{}_'.format(i_file * numtraces_per_file) + r'.hdf5' 
        print('Reading {}...'.format(h5file))
    
        with h5py.File(h5file, 'r') as hdf5_file:
            ptexts[(i_file*numtraces_per_file):((i_file + 1)*numtraces_per_file),:] = hdf5_file['ptexts'][:,:].astype('uint8')
            ctexts[(i_file*numtraces_per_file):((i_file + 1)*numtraces_per_file),:] = hdf5_file['ctexts'][:,:].astype('uint8')
            traces[(i_file*numtraces_per_file):((i_file + 1)*numtraces_per_file),:] = hdf5_file['traces'][:,:].astype('int8')
    
    
    print('Ptexts size:', ptexts.shape)
    print(ptexts)
    print('Ctexts size:', ctexts.shape)
    print(ctexts)
    print('Traces size:', traces.shape)
    print(traces)
    print('Completed')
    
    return ptexts, ctexts, traces


In [ ]:
ptexts, ctexts, traces = ReadData()

### Task 0: Find the AES-128 master key